In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import InterpolatedUnivariateSpline
import seaborn as sns
import pandas as pd
import re
import json
sns.set_style('darkgrid')

In [ ]:
import sys
sys.path.append('../../pytpc')
import pytpc

# Processing ASTAR data into JSON files

The data from ASTAR are just in a whitespace-delimited table. It will be easier to load them for interpolation at runtime if they are instead in JSON files (or something else more machine-readable).

In [ ]:
def read_astar(fp):
    data = []
    for line in fp:
        elmts = line.strip().split()
        if any([re.match(r'\d*\.\d*E[+-]\d*', x) for x in elmts]):
            data.append([float(x) for x in elmts])
    return data

def make_gas_dict(molar, dedx):
    d = {'molar_mass': molar,
         'dedx': dedx}
    return d

## Helium data

This is for pure helium gas

In [ ]:
with open('../data/raw/helium_astar.txt') as f:
    he_dedx = read_astar(f)

he_gas_dict = make_gas_dict(4.002, he_dedx)
with open('../data/gases/helium.json', 'w') as f:
    json.dump(he_gas_dict, f, indent=2)

In [ ]:
hegas = pytpc.gases.InterpolatedGas('helium', 760)
ens = np.logspace(-3, 3, 100)
plt.plot(ens, hegas.energy_loss(ens, 4, 2))
plt.loglog()
plt.xlabel('Energy [MeV]')
plt.ylabel('dE/dx at 760 torr [MeV/m]')

## CO2 data

This is for pure CO2 gas

In [ ]:
with open('../data/raw/co2_astar.txt') as f:
    co2_dedx = read_astar(f)

co2_gas_dict = make_gas_dict(44.01, co2_dedx)
with open('../data/gases/carbon_dioxide.json', 'w') as f:
    json.dump(co2_gas_dict, f, indent=2)

In [ ]:
co2gas = pytpc.gases.InterpolatedGas('carbon_dioxide', 760)
ens = np.logspace(-3, 3, 100)
plt.plot(ens, co2gas.energy_loss(ens, 4, 2))
plt.loglog()
plt.xlabel('Energy [MeV]')
plt.ylabel('dE/dx at 760 torr [MeV/m]')

# Gas Mixtures

## Helium-CO2 90/10

In [ ]:
heco2 = pytpc.gases.InterpolatedGasMixture(760, ('helium', 0.9), ('carbon_dioxide', 0.1))
ens = np.logspace(-3, 3, 100)
plt.plot(ens, heco2.energy_loss(ens, 4, 2))
plt.loglog()
plt.xlabel('Energy [MeV]')
plt.ylabel('dE/dx at 760 torr [MeV/m]')

In [ ]:
ens = np.logspace(-3, 3, 100)
plt.plot(ens, hegas.energy_loss(ens, 4, 2), label='Pure He')
plt.plot(ens, co2gas.energy_loss(ens, 4, 2), label='Pure CO2')
plt.plot(ens, heco2.energy_loss(ens, 4, 2), label='He-CO2 90/10')
plt.loglog()
plt.xlabel('Energy [MeV]')
plt.ylabel('dE/dx at 760 torr [MeV/m]')
plt.legend()